In [68]:
import openai

In [69]:
openai.api_key = 'OPENAI_API_KEY'

In [70]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7Flp61N9gV3EXaefxxTglfNcnMe0A at 0x117d360c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The entire 2020 World Series was played at Globe Life Field in Arlington, Texas due to the COVID-19 pandemic.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683993108,
  "id": "chatcmpl-7Flp61N9gV3EXaefxxTglfNcnMe0A",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 57,
    "total_tokens": 82
  }
}

In [71]:
import openai
import os

In [72]:
class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({'role':'system', 'content': self.prompt})
    
    def ask(self, question):
        try:
            self.messages.append({'role':'user', 'content': question})
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages = self.messages,
                temperature = 0.5,
                max_tokens = 2048,
                top_p = 1,
            )
        except Exception as e:
            print(e)
            return e
        
        message  = response['choices'][0]['message']['content']
        self.messages.append({'role':'assistant', 'content': message})

        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3] # Remove the first round conversation left
        return message

In [73]:

prompt = """
你是一个律师，用中文回答一些法律的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以
"""

conv1 = Conversation(prompt, 2)

question1 = "你是谁？"
print("User : %s" % question1)
print("Assistant : %s\n" % conv1.ask(question1))

User : 你是谁？
Assistant : 我是一个AI语言模型，专门用于回答各种问题，包括法律问题。



In [74]:
question2 = "请问什么是正当防卫？"
print("User : %s" % question2)
print("Assistant : %s\n" % conv1.ask(question2))

User : 请问什么是正当防卫？
Assistant : 正当防卫是指在必要时为了保护自己、他人或者国家公共利益而采取的防御行为。在我国法律中，对于正当防卫的情况，法律规定可以免除或减轻犯罪责任。但是，正当防卫也有限制，必须符合法律规定的情形和条件，否则可能构成违法犯罪行为。



In [75]:
question3 = "那防卫过当呢？"
print("User : %s" % question3)
print("Assistant : %s\n" % conv1.ask(question3))

User : 那防卫过当呢？
Assistant : 防卫过当是指在正当防卫行为中，因过度防卫而超出了必要限度，对袭击者造成了严重伤害或者死亡的行为。在我国法律中，防卫过当是不被允许的，因为它已经超出了必要的防卫范围，可能构成过失犯罪或者故意犯罪。如果行为构成犯罪，防卫人需要承担相应的法律责任。



In [76]:
question4 = "我问你的第一个问题是什么？"
print("User : %s" % question4)
print("Assistant : %s\n" % conv1.ask(question4))

User : 我问你的第一个问题是什么？
Assistant : 你问的第一个问题是："请问什么是正当防卫？"



In [77]:
question5 = "我问你的第一个问题是什么？"
print("User : %s" % question5)
print("Assistant : %s\n" % conv1.ask(question5))

User : 我问你的第一个问题是什么？
Assistant : 你问的第一个问题是："那防卫过当呢？"



In [78]:
class Conversation2:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({'role': 'system', 'content': self.prompt})
    
    def ask(self, question):
        try:
            self.messages.append({'role':'user', 'content': question})
            response = openai.ChatCompletion.create(
                model = 'gpt-3.5-turbo',
                messages = self.messages,
                temperature = 0.5,
                max_tokens = 2048,
                top_p = 1,
            )
        except Exception as e:
            print(e)
            return e

        message = response['choices'][0]['message']['content']
        num_of_tokens = response['usage']['total_tokens']
        self.messages.append({'role':'assistant', 'content':message})

        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message, num_of_tokens

In [79]:
conv2 = Conversation2(prompt, 3)
questions = [question1, question2, question3, question4,question5]
for question in questions:
    answer, num_of_tokens = conv2.ask(question)
    print("询问{%s} 消耗的Token数量是： %d" % (question, num_of_tokens))

询问{你是谁？} 消耗的Token数量是： 113
询问{请问什么是正当防卫？} 消耗的Token数量是： 302
询问{那防卫过当呢？} 消耗的Token数量是： 516
询问{我问你的第一个问题是什么？} 消耗的Token数量是： 549
询问{我问你的第一个问题是什么？} 消耗的Token数量是： 546


In [80]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

conv2 = Conversation2(prompt, 3)
question1 = "你是谁？"
answer1, num_of_tokens = conv2.ask(question1)
print("总共消耗的token数量是 : %d" % (num_of_tokens))

prompt_count = len(encoding.encode(prompt))
question1_count = len(encoding.encode(question1))
answer1_count = len(encoding.encode(answer1))
total_count = prompt_count + question1_count + answer1_count
print("Prompt消耗 %d Token, 问题消耗 %d Token，回答消耗 %d Token，总共消耗 %d Token" % (prompt_count, question1_count, answer1_count, total_count))

总共消耗的token数量是 : 113
Prompt消耗 66 Token, 问题消耗 5 Token，回答消耗 29 Token，总共消耗 100 Token


In [67]:
'''
Author: Hivan Du
mail: doo@hivan.me
LastEditors: Hivan Du
LastEditTime: 2023-05-14 00:51:18
'''
import gradio as gr
prompt = """
你是一个律师，用中文回答一些法律的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以
"""

conv = Conversation(prompt, 10)

def answer(question, history = []):
    history.append(question)
    response = conv.ask(question)
    history.append(response)
    responses = [(u,b) for u, b in zip(history[::2], history[1::2])]
    return responses, history

with gr.Blocks(css="#chatbot{height:300px} .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id='chatbot')
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder = 'Enter text and press enter').style(container=False)
    
    txt.submit(answer, [txt, state], [chatbot, state])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
